# Check the distribution of class in all csv files (Lake SIMC)

In [10]:
import pandas as pd
import os
import glob

# Define the directory containing CSV files
csv_dir = "SIMC_OverlapTiffsWithPP/SIMC.Overlap.csv"  

# Define the target zooplankton classes (The 7 classes that the government cares about.)
target_classes = ["Calanoid_1", "Cyclopoid_1", "Bosmina_1", "Herpacticoida", "Chironomid", "Chydoridae", "Daphnia"]

# Initialize an empty DataFrame to store the combined data
all_data = pd.DataFrame()

# Get a list of all CSV files in the directory
csv_files = glob.glob(os.path.join(csv_dir, "*.csv"))

# Read and combine all CSV files
for file in csv_files:
    df = pd.read_csv(file, usecols=["Class"])  
    all_data = pd.concat([all_data, df], ignore_index=True)

# Count occurrences of each target zooplankton class
class_counts = all_data["Class"].value_counts()

# Filter only the target classes
filtered_counts = class_counts[class_counts.index.isin(target_classes)]

print(filtered_counts)


Class
Calanoid_1       215214
Cyclopoid_1      197982
Bosmina_1          2875
Herpacticoida       607
Daphnia             558
Chydoridae           43
Chironomid           38
Name: count, dtype: int64


Calanoid and Cyclopoid copepods are two of the most ecologically significant and abundant groups of zooplankton.
Considering the computing power and the resources available, a binary classification model for these two classes is possible to develop.

# Data processing
Takes about 1.5 min

## Step 1: Filter and save Calanoid and Cyclopoid data 


In [11]:
# Step 1
import pandas as pd
import os
import glob

print("Step 1: Filter and Save Calanoid and Cyclopoid data...")

csv_dir = "SIMC_OverlapTiffsWithPP/SIMC.Overlap.csv"  
filtered_output = "filtered_zooplankton_data.csv"  
final_output = "merged_filtered_zooplankton_data.csv"  

# Define the target zooplankton classes (Only look at two classes.)
target_classes = ["Calanoid_1", "Cyclopoid_1"]

filtered_data = []

# Get a list of all CSV files in the directory
csv_files = glob.glob(os.path.join(csv_dir, "*.csv"))

# Read and filter all CSV files
for file in csv_files:
    df = pd.read_csv(file)  
    df_filtered = df[df["Class"].isin(target_classes)].copy()  
    if not df_filtered.empty:
        df_filtered.loc[:, "source_file"] = os.path.basename(file)  # add source file column for merge environmental facdtors
        filtered_data.append(df_filtered)  

if filtered_data:
    final_data = pd.concat(filtered_data, ignore_index=True)
    final_data.to_csv(filtered_output, index=False)
    print(f"Step 1 Complete: Filtered data saved to '{filtered_output}'.")
else:
    print("Step 1 Warning: No matching data found in the CSV files.")


Step 1: Filter and Save Calanoid and Cyclopoid data...
Step 1 Complete: Filtered data saved to 'filtered_zooplankton_data.csv'.


## Step 2: Preprocess MasterTable (Remove Duplicates & Filter SIMC)

In [12]:
master_table_path = "MasterTable_AI_FlowCAM.xlsx"  

print("Step 2: Preprocessing MasterTable...")

# Load the MasterTable (environmental data)
master_table = pd.read_excel(master_table_path)

# Ensure consistent column formatting
master_table.rename(columns=lambda x: x.strip().lower(), inplace=True)
master_table["csvfile"] = master_table["csvfile"].astype(str)

# Filter MasterTable to keep only rows where 'csvfile' contains 'SIMC'
master_table = master_table[master_table["csvfile"].str.contains("SIMC", na=False, case=False)]

# Drop duplicates based on 'csvfile' column
master_table = master_table.drop_duplicates(subset=["csvfile"])

print(f"Step 2 Complete: MasterTable filtered and deduplicated. Remaining rows: {len(master_table)}")


Step 2: Preprocessing MasterTable...
Step 2 Complete: MasterTable filtered and deduplicated. Remaining rows: 236


## Step 3: Merge filtered data with preprocessed MasterTable

In [13]:
print("Step 3: Merging filtered data with MasterTable...")

filtered_df = pd.read_csv(filtered_output)

filtered_df["source_file"] = filtered_df["source_file"].astype(str)

# Merge based on 'source_file' (filtered_df) and 'csvfile' (MasterTable)
merged_df = filtered_df.merge(master_table, left_on="source_file", right_on="csvfile", how="left")

merged_df.to_csv(final_output, index=False)

print(f"Step 3 Complete: Merged data saved to '{final_output}'.")

Step 3: Merging filtered data with MasterTable...
Step 3 Complete: Merged data saved to 'merged_filtered_zooplankton_data.csv'.


## Data quality check for merging

In [ ]:
# Number of missing values for each column
t = merged_df.isna().sum().reset_index(name="null_count")
t[t['null_count']>0]

,index,null_count
58,Biovolume..Cylinder.,397474
59,Biovolume..P..Spheroid.,397474
60,Biovolume..Sphere.,397474
61,tifffile,18933
62,csvfile,18933
63,year,18933
64,sam,18933
65,month,18933
66,day,18933
67,rep,18933


In [15]:
merged_df[merged_df['year'].isna()]['source_file'].unique()

array(['20190617_SIMC_238_2mm_rep2_AD_data.csv',
       '20190506_SIMC_017_2mm_rep2_AD_data.csv',
       '20190611_SIMC_218_2mm_rep4_AD_data.csv',
       '20170510_SIMC_095_2mm_rep2_KG_data.csv',
       '20170608_SIMC_177_2mm_rep3_KG_data.csv',
       '20170523_SIMC_133_2mm_rep3_redo_KG_data.csv',
       '20190508_SIMC_047_2mm_rep1_KG_data.csv',
       '20170510_SIMC_095_2mm_rep1_KG_data.csv',
       '20170523_SIMC_133_2mm_rep2_redo_KG_data.csv',
       '20170608_SIMC_179_2mm_rep3_KG_data.csv',
       '20190506_SIMC_017_2mm_rep3_KG_data.csv',
       '20190508_SIMC_047_2mm_rep2_AD_data.csv',
       '20170608_SIMC_177_2mm_rep2_KG_data.csv',
       '20190507_SIMC_033_2mm_rep3_AD_data.csv',
       '20170608_SIMC_179_2mm_rep2_KG_data.csv'], dtype=object)

The environmental factors for these files are not available. So we exclude these data in the analysis. \
In total, exclude about 4.6% data (3.2% are Calanoid, 1.3% are Cyclopoid).


In [16]:
merged_df[merged_df['year'].isna()]['Class'].value_counts() / len(merged_df)

Class
Calanoid_1     0.032299
Cyclopoid_1    0.013521
Name: count, dtype: float64